# Data Wrangling: Uber Data 2014

In [28]:
# Dependencies and Setup
import pandas as pd
import warnings
import time
from datetime import datetime
from pathlib import Path
warnings.filterwarnings("ignore")

In [3]:
# Files to Load

uber_trip_2014 = [
    "uber_resources/uber_trip_2014/uber-raw-data-apr14.csv",
    "uber_resources/uber_trip_2014/uber-raw-data-aug14.csv",
    "uber_resources/uber_trip_2014/uber-raw-data-jul14.csv",
    "uber_resources/uber_trip_2014/uber-raw-data-jun14.csv",
    "uber_resources/uber_trip_2014/uber-raw-data-may14.csv",
    "uber_resources/uber_trip_2014/uber-raw-data-sep14.csv"
]

# Initialize an empty list to store DataFrames
data_frames = []

# Read each CSV file and append its DataFrame to the list
for file_path in uber_trip_2014:
    df = pd.read_csv(file_path)
    data_frames.append(df)

# Concatenate all DataFrames in the list into one DataFrame
raw_data = pd.concat(data_frames, ignore_index=True)
raw_data

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764


## Data Cleaning

In [4]:
# Splitting Date/Time into their own columns
raw_data[['Date', 'Time']] = raw_data['Date/Time'].str.split(expand=True)

raw_data.drop(columns=['Date/Time'], inplace=True)

raw_data["Time"]
del raw_data["Lat"]
del raw_data["Lon"]

raw_data.head()

,Base,Date,Time
0,B02512,4/1/2014,0:11:00
1,B02512,4/1/2014,0:17:00
2,B02512,4/1/2014,0:21:00
3,B02512,4/1/2014,0:28:00
4,B02512,4/1/2014,0:33:00


In [5]:
# Handling data value types:
print(raw_data.info())

print("After data converstion")

data = {'Date': raw_data["Date"]}
datetime = pd.DataFrame(data)

# Convert date columns to datetime dtype
raw_data['Date'] = pd.to_datetime(datetime['Date'])

print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4534327 entries, 0 to 4534326
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Base    object
 1   Date    object
 2   Time    object
dtypes: object(3)
memory usage: 103.8+ MB
None
After data converstion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4534327 entries, 0 to 4534326
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   Base    object        
 1   Date    datetime64[ns]
 2   Time    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 103.8+ MB
None


In [6]:
# Finding and Removing Duplicates 
print(raw_data.count())

print("After removing duplicates:")
non_dupe_data = raw_data.drop_duplicates()
print(non_dupe_data.count())

Base    4534327
Date    4534327
Time    4534327
dtype: int64
After removing duplicates:
Base    937192
Date    937192
Time    937192
dtype: int64


In [7]:
# Handling Missing Values
non_dupe_data = non_dupe_data.dropna()

non_na_data = non_dupe_data.dropna(axis=1)

print(non_na_data.count())

Base    937192
Date    937192
Time    937192
dtype: int64


In [8]:
# Renaming to finalized dataframe
uber_data = non_na_data
uber_data.head()

,Base,Date,Time
0,B02512,2014-04-01,0:11:00
1,B02512,2014-04-01,0:17:00
2,B02512,2014-04-01,0:21:00
3,B02512,2014-04-01,0:28:00
4,B02512,2014-04-01,0:33:00


## 2014 Trips Data

In [9]:
# Finding the Number of trips per day by a base
rides_per_day = uber_data.groupby("Base")["Date"].value_counts()

rides_per_day_df = rides_per_day.reset_index(name="Number of trips")

rides_per_day_df.head()

,Base,Date,Number of trips
0,B02512,2014-09-05,924
1,B02512,2014-04-30,908
2,B02512,2014-04-04,888
3,B02512,2014-06-13,878
4,B02512,2014-05-16,866


## 2014 Summary

In [10]:
# All base names
bases = uber_data["Base"].unique()

# total trips 
num_trips_per_base = rides_per_day_df.groupby("Base")["Number of trips"].sum()

# maximum trips
max_trips_per_base = rides_per_day_df.groupby("Base")["Number of trips"].max()

# minimum trips
min_trips_per_base = rides_per_day_df.groupby("Base")["Number of trips"].min()

# Date Range
start_date = rides_per_day_df["Date"].min()
end_date = rides_per_day_df["Date"].max()

# Total Days
total_days = len(rides_per_day_df["Date"].unique())

# Average Trips per Day
avg_trips_per_day = num_trips_per_base / total_days

In [11]:
# Most Active Day
most_active_dates_idx = rides_per_day_df.groupby('Base')['Number of trips'].idxmax()
most_active_dates = rides_per_day_df.loc[most_active_dates_idx, 'Date']

# Set index to "Base" and rename the index
most_active_dates.index = most_active_dates_idx.index
most_active_dates.index.name = 'Base'

# Least Active Day
least_active_dates_idx = rides_per_day_df.groupby('Base')['Number of trips'].idxmin()
least_active_dates = rides_per_day_df.loc[least_active_dates_idx, 'Date']

# Set index to "Base"
least_active_dates.index = least_active_dates_idx.index
least_active_dates.index.name = 'Base'

In [12]:
uber_summary = pd.DataFrame({
    "Base": bases,
    "Total Trips": num_trips_per_base,
    "Maximum Trips": max_trips_per_base,
    "Minimum Trips": min_trips_per_base,
    "Start Date": start_date,
    "End Date": end_date,
    "Total Days": total_days,
    "Average Trips/Day": avg_trips_per_day,
    "Most Active Day": most_active_dates,
    "Least Active Day": least_active_dates   
})


uber_summary

,Base,Total Trips,Maximum Trips,Minimum Trips,Start Date,End Date,Total Days,Average Trips/Day,Most Active Day,Least Active Day
Base,,,,,,,,,,
B02512,B02512,126052,924,323,2014-04-01,2014-09-30,183,688.808743,2014-09-05,2014-07-05
B02598,B02598,242118,1395,1133,2014-04-01,2014-09-30,183,1323.049180,2014-07-19,2014-04-01
B02617,B02617,234872,1426,984,2014-04-01,2014-09-30,183,1283.453552,2014-08-02,2014-04-20
B02682,B02682,239852,1384,1128,2014-04-01,2014-09-30,183,1310.666667,2014-04-26,2014-05-26
B02764,B02764,94298,1391,113,2014-04-01,2014-09-30,183,515.289617,2014-09-27,2014-07-05


# Data Wrangling: Uber Data 2015

In [16]:
# Files to Load

uber_trip_2015 = "uber_resources/uber_trip_2015/uber-raw-data-janjune-15.csv"

raw_data_2015 = pd.read_csv(uber_trip_2015)

raw_data_2015

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90
...,...,...,...,...
14270474,B02765,2015-05-08 15:43:00,B02765,186
14270475,B02765,2015-05-08 15:43:00,B02765,263
14270476,B02765,2015-05-08 15:43:00,B02765,90
14270477,B02765,2015-05-08 15:44:00,B01899,45


## Data Cleaning

In [17]:
# Splitting Pickup_date into their own columns Date, Time
raw_data_2015[['Date', 'Time']] = raw_data_2015['Pickup_date'].str.split(expand=True)

raw_data_2015.drop(columns=['Pickup_date'], inplace=True)

raw_data_2015["Time"]
del raw_data_2015["Affiliated_base_num"]
del raw_data_2015["locationID"]

raw_data_2015.head()

,Dispatching_base_num,Date,Time
0,B02617,2015-05-17,09:47:00
1,B02617,2015-05-17,09:47:00
2,B02617,2015-05-17,09:47:00
3,B02617,2015-05-17,09:47:00
4,B02617,2015-05-17,09:47:00


In [18]:
# Handling data value types:
print(raw_data_2015.info())

print("After data converstion")

data_2015 = {'Date': raw_data_2015["Date"]}
datetime_2015 = pd.DataFrame(data_2015)

#Convert date columns to datetime dtype
raw_data_2015['Date'] = pd.to_datetime(datetime_2015['Date'])

print(raw_data_2015.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14270479 entries, 0 to 14270478
Data columns (total 3 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Dispatching_base_num  object
 1   Date                  object
 2   Time                  object
dtypes: object(3)
memory usage: 326.6+ MB
None
After data converstion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14270479 entries, 0 to 14270478
Data columns (total 3 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Dispatching_base_num  object        
 1   Date                  datetime64[ns]
 2   Time                  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 326.6+ MB
None


In [19]:
# Finding and removing Duplicates 
print(raw_data_2015.count())

print("After removing duplicates:")
raw_data_2015 = raw_data_2015.drop_duplicates()
print(raw_data_2015.count())

Dispatching_base_num    14270479
Date                    14270479
Time                    14270479
dtype: int64
After removing duplicates:
Dispatching_base_num    4525316
Date                    4525316
Time                    4525316
dtype: int64


In [20]:
# Handling Missing Values
raw_data_2015 = raw_data_2015.dropna()

raw_data_2015 = raw_data_2015.dropna(axis=1)

print(raw_data_2015.count())

Dispatching_base_num    4525316
Date                    4525316
Time                    4525316
dtype: int64


In [21]:
# Renaming to finalized dataframe
uber_data_2015 = raw_data_2015

uber_data_2015.rename(columns={'Dispatching_base_num': 'Base'}, inplace=True)

uber_data_2015.head()

,Base,Date,Time
0,B02617,2015-05-17,09:47:00
10,B02617,2015-05-17,09:48:00
12,B02617,2015-05-17,09:49:00
21,B02617,2015-05-17,09:50:00
29,B02617,2015-05-17,09:51:00


## 2015 Trips Data

In [22]:
# Finding the Number of trips per day by a base
rides_2015 = uber_data_2015.groupby("Base")["Date"].value_counts()

rides_2015_df = rides_2015.reset_index(name="Number of trips")

rides_2015_df

,Base,Date,Number of trips
0,B02512,2015-02-13,2446
1,B02512,2015-02-20,2421
2,B02512,2015-02-19,2134
3,B02512,2015-02-12,2112
4,B02512,2015-02-21,2106
...,...,...,...
1141,B02836,2015-06-08,47
1142,B02836,2015-06-30,45
1143,B02836,2015-06-09,42
1144,B02836,2015-06-18,40


## 2015 Summary

In [23]:
# All base names
bases_2015 = uber_data_2015["Base"].unique()

# total trips 
trips_per_base = rides_2015_df.groupby("Base")["Number of trips"].sum()

# maximum trips
max_trips_base = rides_2015_df.groupby("Base")["Number of trips"].max()

# minimum trips
min_trips_base = rides_2015_df.groupby("Base")["Number of trips"].min()

# Date Range
start_date2 = rides_2015_df["Date"].min()
end_date2 = rides_2015_df["Date"].max()

# Total Days
total_days2 = len(rides_2015_df["Date"].unique())

# Average Trips per Day
avg_trips = trips_per_base / total_days2

In [24]:
# Most Active Day
most_active_idx2 = rides_2015_df.groupby('Base')['Number of trips'].idxmax()
most_active2 = rides_2015_df.loc[most_active_idx2, 'Date']

# Set index to "Base" and rename the index
most_active2.index = most_active_idx2.index
most_active2.index.name = 'Base'

# Least Active Day
least_active_idx2 = rides_2015_df.groupby('Base')['Number of trips'].idxmin()
least_active2 = rides_2015_df.loc[least_active_idx2, 'Date']

# Set index to "Base"
least_active2.index = least_active_idx2.index
least_active2.index.name = 'Base'

In [26]:
u2015_summary = pd.DataFrame({
    "Base": bases_2015,
    "Total Trips": trips_per_base,
    "Maximum Trips": max_trips_base,
    "Minimum Trips": min_trips_base,
    "Start Date": start_date2,
    "End Date": end_date2,
    "Total Days": total_days2,
    "Average Trips/Day": avg_trips,
    "Most Active Day": most_active2,
    "Least Active Day": least_active2   
})

u2015_summary

,Base,Total Trips,Maximum Trips,Minimum Trips,Start Date,End Date,Total Days,Average Trips/Day,Most Active Day,Least Active Day
Base,,,,,,,,,,
B02512,B02617,185902,2446,320,2015-01-01,2015-06-30,181,1027.082873,2015-02-13,2015-04-04
B02598,B02598,655439,11443,1272,2015-01-01,2015-06-30,181,3621.209945,2015-02-13,2015-03-17
B02617,B02682,834533,15129,1331,2015-01-01,2015-06-30,181,4610.679558,2015-02-14,2015-04-14
B02682,B02764,778959,14474,1358,2015-01-01,2015-06-30,181,4303.640884,2015-02-14,2015-03-03
B02764,B02512,1698371,33953,1379,2015-01-01,2015-06-30,181,9383.265193,2015-02-14,2015-03-08
B02765,B02765,353017,7365,929,2015-01-01,2015-06-30,181,1950.370166,2015-02-20,2015-01-27
B02835,B02835,17160,1056,264,2015-01-01,2015-06-30,181,94.806630,2015-06-27,2015-06-08
B02836,B02836,1935,97,39,2015-01-01,2015-06-30,181,10.690608,2015-06-27,2015-06-03


# Data Wrangling: Weather Data

In [38]:
# Read data from the JSON file
path = "weather_resources/weatherData2014-2015.csv"
with open(path, 'r') as csv_file:
    raw_weather_data = pd.read_csv(csv_file)

raw_weather_data.head()

,dt,dt_iso,time,zero,type,lon,temp,visibility,dew_point,feels_like,...,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1388534400,1/1/2014,0:00:00,0,UTC,-74.005973,32.23,10000.0,16.66,20.43,...,32.43,1022,48,19.55,295,0,800,Clear,sky is clear,01n
1,1388538000,1/1/2014,1:00:00,0,UTC,-74.005973,30.78,10000.0,16.16,18.18,...,31.26,1023,50,23.00,285,0,800,Clear,sky is clear,01n
2,1388541600,1/1/2014,2:00:00,0,UTC,-74.005973,30.27,10000.0,14.85,18.57,...,30.94,1024,48,17.27,250,0,800,Clear,sky is clear,01n
3,1388545200,1/1/2014,3:00:00,0,UTC,-74.005973,28.78,10000.0,12.60,16.18,...,29.95,1025,46,20.69,285,0,800,Clear,sky is clear,01n
4,1388548800,1/1/2014,4:00:00,0,UTC,-74.005973,27.64,10000.0,10.63,15.04,...,28.94,1025,44,19.55,305,0,800,Clear,sky is clear,01n


## Data Cleaning

In [39]:
# Unecessary columns removal 

del raw_weather_data["dt"]
del raw_weather_data["zero"]
del raw_weather_data["type"]
del raw_weather_data["lon"]
del raw_weather_data["visibility"]
del raw_weather_data["dew_point"]
del raw_weather_data["pressure"]
del raw_weather_data["wind_deg"]
del raw_weather_data["weather_main"]
del raw_weather_data["weather_icon"]
del raw_weather_data["weather_id"]

# Convert 'dt_iso' column to datetime format
raw_weather_data['dt_iso'] = pd.to_datetime(raw_weather_data['dt_iso'])

# Rename columns
raw_weather_data.rename(columns={'dt_iso': 'Date'}, inplace=True)
raw_weather_data.rename(columns={'time': 'time_HMS'}, inplace=True)
raw_weather_data.rename(columns={'clouds_all': 'Cloudiness'}, inplace=True)

raw_weather_data.info()
raw_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18511 entries, 0 to 18510
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 18511 non-null  datetime64[ns]
 1   time_HMS             18511 non-null  object        
 2   temp                 18511 non-null  float64       
 3   feels_like           18511 non-null  float64       
 4   temp_min             18511 non-null  float64       
 5   temp_max             18511 non-null  float64       
 6   humidity             18511 non-null  int64         
 7   wind_speed           18511 non-null  float64       
 8   Cloudiness           18511 non-null  int64         
 9   weather_description  18511 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 1.4+ MB


,Date,time_HMS,temp,feels_like,temp_min,temp_max,humidity,wind_speed,Cloudiness,weather_description
0,2014-01-01,0:00:00,32.23,20.43,30.16,32.43,48,19.55,0,sky is clear
1,2014-01-01,1:00:00,30.78,18.18,30.15,31.26,50,23.00,0,sky is clear
2,2014-01-01,2:00:00,30.27,18.57,28.36,30.94,48,17.27,0,sky is clear
3,2014-01-01,3:00:00,28.78,16.18,26.56,29.95,46,20.69,0,sky is clear
4,2014-01-01,4:00:00,27.64,15.04,26.24,28.94,44,19.55,0,sky is clear


In [40]:
# Finding and removing Duplicates 
print(raw_weather_data.count())

print("After removing duplicates:")
raw_weather_data = raw_weather_data.drop_duplicates()
print(raw_weather_data.count())

Date                   18511
time_HMS               18511
temp                   18511
feels_like             18511
temp_min               18511
temp_max               18511
humidity               18511
wind_speed             18511
Cloudiness             18511
weather_description    18511
dtype: int64
After removing duplicates:
Date                   18511
time_HMS               18511
temp                   18511
feels_like             18511
temp_min               18511
temp_max               18511
humidity               18511
wind_speed             18511
Cloudiness             18511
weather_description    18511
dtype: int64


In [41]:
# Handling Missing Values
raw_weather_data = raw_weather_data.dropna()

raw_weather_data = raw_weather_data.dropna(axis=1)

In [42]:
# Renaming to finalized dataframe
weather_data = raw_weather_data
weather_data

,Date,time_HMS,temp,feels_like,temp_min,temp_max,humidity,wind_speed,Cloudiness,weather_description
0,2014-01-01,0:00:00,32.23,20.43,30.16,32.43,48,19.55,0,sky is clear
1,2014-01-01,1:00:00,30.78,18.18,30.15,31.26,50,23.00,0,sky is clear
2,2014-01-01,2:00:00,30.27,18.57,28.36,30.94,48,17.27,0,sky is clear
3,2014-01-01,3:00:00,28.78,16.18,26.56,29.95,46,20.69,0,sky is clear
4,2014-01-01,4:00:00,27.64,15.04,26.24,28.94,44,19.55,0,sky is clear
...,...,...,...,...,...,...,...,...,...,...
18506,2015-12-31,19:00:00,47.62,42.62,47.01,48.34,59,11.41,100,overcast clouds
18507,2015-12-31,20:00:00,47.79,41.81,47.19,48.45,57,14.99,100,overcast clouds
18508,2015-12-31,21:00:00,47.61,41.58,47.26,48.11,59,14.99,100,overcast clouds
18509,2015-12-31,22:00:00,46.98,42.19,46.47,47.66,55,10.29,100,overcast clouds


## Weather Summary

In [44]:
# Copy data and delete time column
weather_data_c = weather_data.copy()
del weather_data_c["time_HMS"]

In [45]:
# Function to find the mode of a series
def mode(series):
    return series.value_counts().index[0]

weather_summary = weather_data_c.groupby('Date').agg({
    'temp': 'mean',
    'feels_like': 'mean',
    'temp_min': 'min',
    'temp_max': 'max',
    'humidity': 'mean',
    'wind_speed': 'mean',
    'Cloudiness': 'mean',
    'weather_description': mode
}).reset_index()


new_column_names = {
    'Date': 'Date',
    'temp': 'Mean_Temp',
    'feels_like': 'Mean_Feels_Like',
    'temp_min': 'Min_Temp',
    'temp_max': 'Max_Temp',
    'humidity': 'Mean_Humidity',
    'wind_speed': 'Mean_Wind_Speed',
    'Cloudiness': 'Mean_Cloudiness',
    'weather_description': 'Weather_Description'
}

weather_summary.rename(columns=new_column_names, inplace=True)

# Now summary_data contains summarized data for each date
weather_summary

,Date,Mean_Temp,Mean_Feels_Like,Min_Temp,Max_Temp,Mean_Humidity,Mean_Wind_Speed,Mean_Cloudiness,Weather_Description
0,2014-01-01,28.941667,19.074167,22.96,34.95,48.958333,12.807917,13.125000,sky is clear
1,2014-01-02,29.206538,18.154615,22.96,34.14,71.576923,16.800000,93.846154,overcast clouds
2,2014-01-03,16.499286,3.899286,10.04,26.55,75.642857,20.561786,75.714286,snow
3,2014-01-04,15.930417,5.985417,6.76,28.11,50.083333,8.940000,3.125000,sky is clear
4,2014-01-05,30.174138,22.742759,18.95,37.11,76.068966,8.063793,59.310345,sky is clear
...,...,...,...,...,...,...,...,...,...
725,2015-12-27,52.473333,51.013333,46.02,62.58,84.000000,7.580370,100.000000,mist
726,2015-12-28,43.968750,37.787500,39.29,61.29,56.500000,13.561250,31.666667,sky is clear
727,2015-12-29,42.106571,34.629714,33.96,49.96,82.914286,15.026571,100.000000,mist
728,2015-12-30,43.105417,39.732083,39.45,48.65,85.166667,5.863750,83.333333,mist


# Output Results

In [46]:
# 2014 Data
uber_data.to_csv('data/uber_data_aprsep_2014.csv', index=False)

rides_per_day_df.to_csv('data/rides_per_day_aprsep_2014.csv', index=False)

uber_summary.to_csv('data/uber_summary_aprsep_2014.csv', index=False)

# 2015 data
uber_data_2015.to_csv('data/uber_data_janjune_2015.csv', index=False)

rides_2015_df.to_csv('data/rides_per_day_janjune_2015.csv', index=False)

u2015_summary.to_csv('data/uber_summary_janjune_2015.csv', index=False)

# Weather data
weather_data.to_csv('data/weather_data_2014-2015.csv', index=False)

weather_summary.to_csv('data/weather_daily_summary_2014-2015.csv', index=False)